In [ ]:
# RAG Retrieval Auguments Generated

# [Retrieval 설명](https://python.langchain.com/docs/modules/data_connection/)
# [Document loaders 종류](https://python.langchain.com/docs/integrations/document_loaders)

# Loader: 소스에서 데이터를 추출하고 랭체인에 가져다 주는 코드

#  Load -> Text Splitting -> Text embedding
# 문서를 embedd 저장하거나 언어 모델에 주고싶다면 Load 할때 하나의 문서로 너무 크게 가져오기 때문에 나누는 작업(split)을 해야한다.
# 그러면 질문에 답해야할 때 필요한 "파일의 부분들"만을 전달할 수 있다.
# 전체 문서나 챕터를 LLM에게 주는게 아니라, 딱 필요한 부분만 사용할 수 있다.
from langchain.document_loaders import TextLoader

loader = TextLoader("./files/chapter_one.txt")

loader.load()

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./files/chapter_one.pdf")

loader.load()

In [ ]:
# [Unstructured File](https://python.langchain.com/docs/integrations/document_loaders/unstructured_file)

# Unstructured는 텍스트 파일, 파워포인트, HTML, PDF, 이미지 등의 파일을 불러오는 기능을 지원합니다.
# 파일 종류를 신경쓰지 않아서 되서 편하다.

# RecursiveCharacterTextSplitter 문장의 끝이나 문단의 끝부분마다 끊어준다.
# chunk_size를 작게하면 중간의 문장을 잘라내서 잘라진 부분의 의미가 달라질 수 있어서 유효성이 안좋다.
# chunk_overlap을 사용하면 문장이나 문단을 분할할때 앞 조각 일부분을 가져와서 문장 또는 문자의 의미가 바뀌지 않게한다.

from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # 얼마나 큰 사이즈로 나눌지 결정
    chunk_overlap=50,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load()
# case 1)
# splitter.split_documents(docs)

# case 2)
result = loader.load_and_split(
    text_splitter=splitter,
)
result

In [15]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

# tickon은 openai에 의해 만들어졌다. 모델과 같은 방법으로 사용한다.

# CharacterTextSplitter 특정 문자열을 찾아 그곳부터 분할한다.(separator)
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",  # 특정 문자열을 찾아 그곳부터 분할한다.
    chunk_size=200,  # 얼마나 큰 사이즈로 나눌지 결정
    chunk_overlap=50,  # 문장이나 문단을 불할할때 앞조각 일부를 가져와 문장의 시작부분에 넣어준다.
    # length_function=len,  # 명시적으로 입력하지 않아도 기본 설정된다.
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load()

result = loader.load_and_split(
    text_splitter=splitter,
)
# 기본적으로 모든 splitter는 텍스트의 length를 계산하여 한 덩어리(chunk)의 크기를 알아낸다.
# 그작업에 파이썬 표준 라이브러리가 지원하는 표준 len함수를 사용한다.
# 사용자 본인만의 len 함수를 명시적으로 설정할 수 있다.

# LLM은 Token을 같은 방법으로 세어주지 않는다.
# [tokenizer](https://platform.openai.com/tokenizer)

Created a chunk of size 218, which is longer than the specified 200
Created a chunk of size 212, which is longer than the specified 200
Created a chunk of size 254, which is longer than the specified 200
Created a chunk of size 241, which is longer than the specified 200
Created a chunk of size 215, which is longer than the specified 200
Created a chunk of size 215, which is longer than the specified 200
Created a chunk of size 372, which is longer than the specified 200
Created a chunk of size 430, which is longer than the specified 200
Created a chunk of size 411, which is longer than the specified 200
Created a chunk of size 237, which is longer than the specified 200
Created a chunk of size 387, which is longer than the specified 200
Created a chunk of size 356, which is longer than the specified 200
Created a chunk of size 508, which is longer than the specified 200
Created a chunk of size 207, which is longer than the specified 200
Created a chunk of size 219, which is longer tha